In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('_train.csv', encoding='iso-8859-1')

In [3]:
df.head(20)

,tag1,tag2,tag3
0,Melbourne,NP B-LOC,B-LOC
1,(,Fpa,O
2,Australia,NP B-LOC,B-LOC
3,),Fpt,O
4,-,Fc,O
5,25,Z O,O
6,may,NC O,O
7,(,Fpa,O
8,EFE,NC B-,B-ORG
9,),Fpt,O


In [4]:
train = df.iloc[:,0:3]

In [52]:
train.head()

,tag1,tag2,tag3
0,Melbourne,NP B-LOC,B-LOC
1,(,Fpa,O
2,Australia,NP B-LOC,B-LOC
3,),Fpt,O
4,-,Fc,O


In [5]:
train.isnull().sum()

tag1    8322
tag2    8322
tag3    8322
dtype: int64

In [46]:
_train = df.iloc[:,2]

In [47]:
_train.head()

0         O
1     B-LOC
2         O
3         O
4         O
Name:  B-LOC, dtype: object

In [6]:
train.groupby('tag3').size().reset_index(name='counts')

,tag3,counts
0,B-LOC,4913
1,B-MISC,2173
2,B-ORG,7390
3,B-PER,4321
4,I-LOC,1891
5,I-MISC,3212
6,I-ORG,4992
7,I-PER,3903
8,O,231920


In [7]:
train = train.fillna(method='ffill')

In [82]:
train

,tag1,tag2,tag3
0,Melbourne,NP B-LOC,B-LOC
1,(,Fpa,O
2,Australia,NP B-LOC,B-LOC
3,),Fpt,O
4,-,Fc,O
5,25,Z O,O
6,may,NC O,O
7,(,Fpa,O
8,EFE,NC B-,B-ORG
9,),Fpt,O


In [8]:
words = list(set(train["tag1"].values))

In [9]:
words

['afrobrasileï¿½a',
 'sovieticas',
 'TRATABAN',
 'excedera',
 '40.802',
 'Kroon',
 'Mano',
 '608',
 'QUARANTA',
 'kilometros',
 'tejado',
 'SELECTIVIDAD',
 'REPUBLICA',
 'marcado',
 '11.2',
 'lineas',
 'noticias',
 'generos',
 'sociolaboral',
 'descender',
 'Karel',
 'XX',
 'necesarios',
 '16-45',
 'tope',
 'Bugallo',
 'lm',
 'expertos',
 '9.83',
 'posiciones',
 'expidieron',
 'integrado',
 'cajon',
 'ofrece',
 'Chisholm',
 'dirimir',
 'Riojana',
 'condeno',
 'Duran.',
 'patronato',
 '1.158-25',
 'esas',
 'influjo',
 'Accionistas',
 'contribuciones',
 'bulgaro',
 'transmitio',
 'Grease',
 '62',
 '7155',
 'fluidas',
 'Anuncio',
 'derrumbe',
 'billon',
 'desventaja',
 '7.4',
 '40758',
 'corresponsal',
 '2263',
 'Norman',
 'capacitaran',
 'lastre',
 'aportadas',
 'concluido',
 'denunciara',
 'atraque',
 'Balonmano',
 'domiciliaria',
 'FIYI',
 'trasladada',
 'usurpada',
 'Z.',
 'mortales',
 'caso',
 'Mayor-',
 'mexicanas',
 'abductor',
 'encierro',
 'aï¿½os',
 'Hacienda-',
 'Duvalier',
 'a

In [10]:
len_words = len(words)
len_words

25661

In [11]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
    
    def get_next(self):
        try:
            #s = self.data[self.data["Sentence #"] == "Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return train["tag1"].values.tolist(), train["tag2"].values.tolist(), train["tag3"].values.tolist()    
        except:
            self.empty = True
            return None, None, None

In [12]:
getter = SentenceGetter(train)

In [13]:
sent, pos, tag = getter.get_next()

In [14]:
print(tag)

[' B-LOC', ' O', ' B-LOC', ' O', ' O', ' O', ' O', ' O', ' B-ORG', ' O', ' O', ' O', ' O', ' O', ' O', ' B-PER', ' I-PER', ' I-PER', ' I-PER', ' O', ' B-PER', ' I-PER', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' B-MISC', ' O', ' O', ' O', ' O', ' O', ' B-PER', ' I-PER', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' B-ORG', ' I-ORG', ' O', ' O', ' O', ' B-LOC', ' O', ' B-LOC', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' B-MISC', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O',

### Memorización

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin


class MemoryTagger(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y):
        '''
        Expects a list of words as X and a list of tags as y.
        '''
        voc = {}
        self.tags = []
        for x, t in zip(X, y):
            if t not in self.tags:
                self.tags.append(t)
            if x in voc:
                if t in voc[x]:
                    voc[x][t] += 1
                else:
                    voc[x][t] = 1
            else:
                voc[x] = {t: 1}
        self.memory = {}
        for k, d in voc.items():
            self.memory[k] = max(d, key=d.get)
    
    def predict(self, X, y=None):
        '''
        Predict the the tag from memory. If word is unknown, predict 'O'.
        '''
        return [self.memory.get(x, 'O') for x in X]

In [16]:
tagger = MemoryTagger()

In [17]:
tagger.fit(sent, tag)

In [18]:
print(tagger.predict(sent))

[' B-LOC', ' O', ' B-LOC', ' O', ' O', ' O', ' O', ' O', ' B-ORG', ' O', ' O', ' O', ' O', ' O', ' O', ' B-PER', ' I-ORG', ' O', ' B-ORG', ' O', ' B-PER', ' I-PER', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' B-MISC', ' O', ' O', ' O', ' O', ' O', ' B-PER', ' I-ORG', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' B-ORG', ' I-ORG', ' O', ' O', ' O', ' B-LOC', ' O', ' B-LOC', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' B-MISC', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O', ' O

In [19]:
tagger.tags

[' B-LOC',
 ' O',
 ' B-ORG',
 ' B-PER',
 ' I-PER',
 ' B-MISC',
 ' I-ORG',
 ' I-LOC',
 ' I-MISC']

In [22]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

In [23]:
words = train["tag1"].values.tolist()
tags = train["tag3"].values.tolist()

In [24]:
pred = cross_val_predict(estimator=MemoryTagger(), X=words, y=tags, cv=5)

In [25]:
report = classification_report(y_pred=pred, y_true=tags)

/home/diegojavier/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [26]:
print(report)

              precision    recall  f1-score   support

       B-LOC       0.67      0.64      0.66      4914
      B-MISC       0.59      0.38      0.46      2173
       B-ORG       0.82      0.70      0.76      7390
       B-PER       0.75      0.51      0.61      4321
       I-LOC       0.54      0.25      0.35      1891
      I-MISC       0.46      0.14      0.21      3215
       I-ORG       0.63      0.25      0.36      4992
       I-PER       0.63      0.39      0.48      3903
           O       0.98      0.94      0.96    240238
           O       0.00      0.00      0.00         0

   micro avg       0.88      0.88      0.88    273037
   macro avg       0.61      0.42      0.48    273037
weighted avg       0.94      0.88      0.91    273037



### Enfoque de machine learning

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
def feature_map(word):
    '''Simple feature map.'''
    return np.array([word.istitle(), word.islower(), word.isupper(), len(word),
                     word.isdigit(),  word.isalpha()])

In [30]:
words = [feature_map(w) for w in train["tag1"].values.tolist()]

In [32]:
pred = cross_val_predict(RandomForestClassifier(n_estimators=20),
                         X=words, y=tags, cv=5)

In [33]:
report = classification_report(y_pred=pred, y_true=tags)
print(report)

/home/diegojavier/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

       B-LOC       0.22      0.52      0.31      4914
      B-MISC       0.19      0.02      0.04      2173
       B-ORG       0.43      0.55      0.48      7390
       B-PER       0.23      0.36      0.28      4321
       I-LOC       0.00      0.00      0.00      1891
      I-MISC       0.18      0.01      0.02      3215
       I-ORG       0.27      0.04      0.07      4992
       I-PER       0.55      0.04      0.08      3903
           O       0.98      0.99      0.98    240238

   micro avg       0.90      0.90      0.90    273037
   macro avg       0.34      0.28      0.25    273037
weighted avg       0.89      0.90      0.89    273037



## Probando la data de prueba a

In [69]:
#Cargando el data set de prueba A
da = pd.read_csv('_testa.csv', encoding='iso-8859-1')

In [70]:
testa = da.iloc[:,0:3]

In [71]:
testa.groupby('tag3').size().reset_index(name='counts')

,tag3,counts
0,B-LOC,984
1,B-MISC,445
2,B-ORG,1700
3,B-PER,1222
4,I-LOC,337
5,I-MISC,654
6,I-ORG,1366
7,I-PER,859
8,O,45356


In [72]:
testa = testa.fillna(method='ffill')

In [75]:
words = list(set(testa["tag1"].values))
len_words = len(words)

In [76]:
len_words

9517

In [77]:
#Separando las pàlabras y las etiquetas
words = testa["tag1"].values.tolist()
tags = testa["tag3"].values.tolist()

In [78]:
#Prediciendo los valores del datasetA
pred = cross_val_predict(estimator=MemoryTagger(), X=words, y=tags, cv=5)

In [86]:
report = classification_report(y_pred=pred, y_true=tags)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.15      0.04      0.06       985
      B-MISC       0.68      0.09      0.16       445
       B-ORG       0.40      0.50      0.45      1700
       B-PER       0.23      0.73      0.36      1222
       I-LOC       0.00      0.00      0.00       337
      I-MISC       0.13      0.00      0.01       654
       I-ORG       0.30      0.05      0.09      1366
       I-PER       0.17      0.01      0.02       859
           O       0.97      0.99      0.98     47269

   micro avg       0.89      0.89      0.89     54837
   macro avg       0.34      0.27      0.23     54837
weighted avg       0.87      0.89      0.87     54837



### Utilizando RandomForestClassifier

In [83]:
#Separando las pàlabras y las etiquetas
words = [feature_map(w) for w in testa["tag1"].values.tolist()]

In [84]:
#Prediciendo los valores del datasetA
pred = cross_val_predict(RandomForestClassifier(n_estimators=20),
                         X=words, y=tags, cv=5)

In [85]:
report = classification_report(y_pred=pred, y_true=tags)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.15      0.04      0.06       985
      B-MISC       0.68      0.09      0.16       445
       B-ORG       0.40      0.50      0.45      1700
       B-PER       0.23      0.73      0.36      1222
       I-LOC       0.00      0.00      0.00       337
      I-MISC       0.13      0.00      0.01       654
       I-ORG       0.30      0.05      0.09      1366
       I-PER       0.17      0.01      0.02       859
           O       0.97      0.99      0.98     47269

   micro avg       0.89      0.89      0.89     54837
   macro avg       0.34      0.27      0.23     54837
weighted avg       0.87      0.89      0.87     54837



/home/diegojavier/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Probando la data de prueba b

In [89]:
#Cargando el data set de prueba B
db = pd.read_csv('_testb.csv', encoding='iso-8859-1')

In [90]:
testb = db.iloc[:,0:3]

In [91]:
testb.groupby('tag3').size().reset_index(name='counts')

,tag3,counts
0,B-LOC,1084
1,B-MISC,339
2,B-ORG,1400
3,B-PER,735
4,I-LOC,325
5,I-MISC,557
6,I-ORG,1104
7,I-PER,634
8,O,45355


In [92]:
testb = testb.fillna(method='ffill')

In [93]:
words = list(set(testb["tag1"].values))
len_words = len(words)
len_words

8976

In [94]:
#Separando las pàlabras y las etiquetas
words = testb["tag1"].values.tolist()
tags = testb["tag3"].values.tolist()

In [95]:
#Prediciendo los valores del datasetB
pred = cross_val_predict(estimator=MemoryTagger(), X=words, y=tags, cv=5)

In [96]:
report = classification_report(y_pred=pred, y_true=tags)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.73      0.53      0.61      1084
      B-MISC       0.67      0.34      0.45       339
       B-ORG       0.85      0.61      0.71      1400
       B-PER       0.76      0.34      0.47       735
       I-LOC       0.42      0.15      0.22       325
      I-MISC       0.34      0.07      0.11       558
       I-ORG       0.61      0.17      0.26      1104
       I-PER       0.60      0.27      0.38       634
           O       0.98      0.88      0.93     46870
           O       0.00      0.00      0.00         0

   micro avg       0.82      0.82      0.82     53049
   macro avg       0.60      0.34      0.41     53049
weighted avg       0.94      0.82      0.87     53049



/home/diegojavier/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### Utilizando RandomForestClassifier

In [97]:
#Separando las pàlabras y las etiquetas
words = [feature_map(w) for w in testb["tag1"].values.tolist()]

In [98]:
#Prediciendo los valores del datasetB
pred = cross_val_predict(RandomForestClassifier(n_estimators=20),
                         X=words, y=tags, cv=5)

In [99]:
report = classification_report(y_pred=pred, y_true=tags)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.29      0.38      0.33      1084
      B-MISC       0.33      0.04      0.07       339
       B-ORG       0.35      0.69      0.47      1400
       B-PER       0.22      0.23      0.22       735
       I-LOC       0.00      0.00      0.00       325
      I-MISC       0.20      0.01      0.01       558
       I-ORG       0.13      0.06      0.08      1104
       I-PER       0.50      0.02      0.04       634
           O       0.98      0.99      0.98     46870

   micro avg       0.91      0.91      0.91     53049
   macro avg       0.33      0.27      0.24     53049
weighted avg       0.89      0.91      0.89     53049



/home/diegojavier/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
